<a href="https://colab.research.google.com/github/roshk10/startup-profit-prediction/blob/main/notebooks/startup_profit_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

file_path = "/content/drive/MyDrive/ML_Projects startup-profit-prediction data/50_Startups.csv"
df = pd.read_csv(file_path)

df.head()


,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


['Roshan (1)',
 'Roshan',
 'VICTORY VORTEX (1).pptx',
 'VICTORY VORTEX.pptx',
 'VICTORY VORTEX1.pptx',
 'ROSHAN RAGURAMAN 312424104205.pdf',
 'Revathy',
 'Colab Notebooks',
 'Cognifyz Technologies Internship_FinalTaskSubmission_Roshan Raguraman.zip',
 'Payment transaction image.jpg',
 'infosys certificates (1).pdf',
 'infosys certificates.pdf',
 'Screenshot_20250801_153131_Chrome.jpg',
 'II CSE-D',
 'D section',
 'ROSHAN RAGURAMAN_RESUME .pdf',
 'RoshanRaguraman_Datascienceandanalytics_Certificate (2).pdf',
 'RoshanRaguraman_Datascienceandanalytics_Certificate (1).pdf',
 'RoshanRaguraman_Datascienceandanalytics_Certificate.pdf',
 'RoshanRaguraman_Datascienceandanalytics_Proof (1).pdf',
 'RoshanRaguraman_Datascienceandanalytics_Proof.pdf',
 'ROSHAN RAGURAMAN_RESUME -1.pdf',
 'II Year CSE D',
 'ROSHAN RAGURAMAN NPTEL Certificate.pdf',
 'ieee card',
 'PEP CERTIFICATION.pdf',
 'infosys certificateint.pdf',
 'skillrack proof.pdf',
 'ML_Projects startup-profit-prediction data']